In [88]:
import pandas as pd
import statsmodels.api as sm
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [89]:

data =[
    #['plan_name', 'total_fup', 'main_bucket_fup', 'main_bucket_speed', 'video_fup', 'video_speed', 'social_fup', 'social_speed', 'unl_ind', 'actual_adpu'])
    ['tuneunl', 105, 105, 6, 0, 6, 0, 6, 1, 48.6],
    ['bebas55+', 100, 100, 18, 0, 18, 0, 18, 1, 61],
    ['bebas55', 80, 80, 18, 0, 18, 0, 18, 1, 55],
    ['fokus40+', 86, 86, 8, 0, 8, 0, 8, 1, 47.2],
    ['fokus40', 66, 66, 8, 0, 8, 0, 8, 1, 41.5],
    ['fokus33+', 64, 64, 4, 0, 4, 0, 4, 1, 34.2],
    ['fokus33', 54, 54, 4, 0, 4, 0, 4, 1, 30.5],
    # ['zerofupspeed', 0, 0, 0, 0, 0, 0, 0, 0, 0],
    # ['zerofup', 0, 0, 18, 0, 18, 0, 18, 0, 0],
    # ['zerospeed', 100, 100, 0, 0, 0, 0, 0, 1, 0],
    ['hi15', 10, 10, 18, 0, 1, 0, 18, 0, 6.7],
    ['hi25', 25, 25, 18, 0, 1, 0, 18, 0, 15.4],
    ['hi35', 35, 35, 18, 0, 18, 0, 18, 0, 21.2],
    ['m10-cun', 170, 10, 18, 60, 1, 100, 18, 1, 26.9],
    ['m10-value', 150, 10, 18, 40, 18, 100, 18, 1, 27.6],
    ['m6-cun', 121, 6, 18, 15, 1, 100, 18, 1, 21],
    ['m6-value', 112, 6, 18, 6, 18, 100, 18, 1, 21],
    ['bestbasic28', 30, 30, 3, 0, 3, 0, 3, 1, 19.9]
]

df = pd.DataFrame(data=data, columns = ['plan_name', 'total_fup', 'main_bucket_fup', 'main_bucket_speed', 'video_fup', 
                                        'video_speed', 'social_fup', 'social_speed', 'unl_ind', 'actual_adpu'])
# df['zerorised_ind'] = [1 if fup == 0 or speed == 0 else 0 for fup, speed in zip(df['main_bucket_fup'], df['main_bucket_speed'])]
df['very_low_fup_ind'] = [1 if fup <= 10 else 0 for fup in df['total_fup']]
df['low_fup_ind'] = [1 if fup <= 30 else 0 for fup in df['total_fup']]
df['high_fup_ind'] = [1 if fup >= 80 else 0 for fup in df['total_fup']]
df['separated_video_fup_ind'] = [1 if video_fup != 0 else 0 for video_fup in df['video_fup']]
df['separated_social_fup_ind'] = [1 if social_fup != 0 else 0 for social_fup in df['social_fup']]
df['slower_video_speed_ind'] = [1 if video_speed != 0 and video_speed < main_bucket_speed else 0 
                                    for video_speed, main_bucket_speed in zip(df['video_speed'], df['main_bucket_speed'])]
#df['slower_social_speed_ind'] = [1 if social_speed != 0 and social_speed < main_bucket_speed else 0 
 #                                   for social_speed, main_bucket_speed in zip(df['social_speed'], df['main_bucket_speed'])]

In [90]:
core_variables = ['total_fup', 'main_bucket_fup', 'main_bucket_speed', 'video_fup', 'video_speed', 'social_fup', 
             'social_speed', 'unl_ind']
# variables = ['total_fup', 'main_bucket_fup', 'main_bucket_speed', 'video_fup', 'video_speed', 'social_fup', 
#              'social_speed', 'unl_ind', 'very_low_fup_ind', 'low_fup_ind', 'high_fup_ind', 'separated_video_fup_ind', 
#              'separated_social_fup_ind', 'slower_video_speed_ind']
variables = ['main_bucket_fup', 'main_bucket_speed', 'video_fup', 'video_speed', 'social_fup', 
             'social_speed', 'unl_ind', 'very_low_fup_ind', 'low_fup_ind', 'high_fup_ind', 'separated_video_fup_ind', 
             'separated_social_fup_ind', 'slower_video_speed_ind']

In [91]:
df_exog = sm.add_constant(df[variables])
df_endog = df[['actual_adpu']]

In [92]:
poisson_model = sm.GLM(df_endog, df_exog, family=sm.families.Poisson())

In [93]:
poisson_results = poisson_model.fit()

In [94]:
print(poisson_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:            actual_adpu   No. Observations:                   15
Model:                            GLM   Df Residuals:                        5
Model Family:                 Poisson   Df Model:                            9
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -39.109
Date:                Thu, 21 Sep 2023   Deviance:                      0.63140
Time:                        12:17:42   Pearson chi2:                    0.636
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

In [71]:
poisson_results.predict()

array([49.29308347, 62.84162382, 54.53697167, 45.12832104, 38.57063991,
       34.65913371, 32.28787874,  6.7       , 14.71765236, 21.88234764,
       27.59016343, 26.50592697, 20.99218421, 21.41172539, 20.58234764])

In [72]:
df_final = df_exog
df_final['actual_adpu'] = df_endog['actual_adpu']
df_final['predicted_adpu'] = poisson_results.predict()
df_final['error'] = df_final['predicted_adpu'] - df_final['actual_adpu']
df_final['plan_name'] = df['plan_name']
rmse = sum(df_final['error']**2)**1/2
print(rmse)

11.803175909981846


In [73]:
df_final

,const,main_bucket_fup,main_bucket_speed,video_fup,video_speed,social_fup,social_speed,unl_ind,very_low_fup_ind,low_fup_ind,high_fup_ind,separated_video_fup_ind,separated_social_fup_ind,slower_video_speed_ind,actual_adpu,predicted_adpu,error,plan_name
0,1.00,105,6,0,6,0,6,1,0,0,1,0,0,0,48.60,49.29,0.69,tuneunl
1,1.00,100,18,0,18,0,18,1,0,0,1,0,0,0,61.00,62.84,1.84,bebas55+
2,1.00,80,18,0,18,0,18,1,0,0,1,0,0,0,55.00,54.54,-0.46,bebas55
3,1.00,86,8,0,8,0,8,1,0,0,1,0,0,0,47.20,45.13,-2.07,fokus40+
4,1.00,66,8,0,8,0,8,1,0,0,0,0,0,0,41.50,38.57,-2.93,fokus40
5,1.00,64,4,0,4,0,4,1,0,0,0,0,0,0,34.20,34.66,0.46,fokus33+
6,1.00,54,4,0,4,0,4,1,0,0,0,0,0,0,30.50,32.29,1.79,fokus33
7,1.00,10,18,0,1,0,18,0,1,1,0,0,0,1,6.70,6.70,0.00,hi15
8,1.00,25,18,0,1,0,18,0,0,1,0,0,0,1,15.40,14.72,-0.68,hi25
9,1.00,35,18,0,18,0,18,0,0,0,0,0,0,0,21.20,21.88,0.68,hi35


In [74]:
total_fup_range = range(10, 110, 10)
main_bucket_speed_range = range(1, 19, 1)

possible_combinations = []
for total_fup in total_fup_range:
    for main_bucket_speed in main_bucket_speed_range:
        possible_combinations += [[total_fup, main_bucket_speed]]

In [78]:
df_possible_combinations = pd.DataFrame(data=possible_combinations, columns = ['total_fup', 'main_bucket_speed'])
df_possible_combinations['main_bucket_fup'] = df_possible_combinations['total_fup']
df_possible_combinations['video_fup'] = 0.0
df_possible_combinations['video_speed'] = df_possible_combinations['main_bucket_speed']
df_possible_combinations['social_fup'] = 0.0
df_possible_combinations['social_speed'] = df_possible_combinations['main_bucket_speed']
df_possible_combinations['unl_ind'] = 1.0
df_possible_combinations = df_possible_combinations[core_variables]

In [79]:
# add new plan
# ['total_fup', 'main_bucket_fup', 'main_bucket_speed', 'video_fup', 'video_speed', 'social_fup', 'social_speed', 'unl_ind')
new_plan = [
            [145, 30, 18, 50, 6, 50, 6, 1], #all35
            [145, 30, 18, 50, 6, 50, 6, 0], #all35-non-unl
            # [500, 500, 6, 0, 6, 0, 6, 1],
            [35, 35, 18, 0, 18, 0, 18, 0], #hi35
            [50, 50, 18, 0, 18, 0, 18, 0], #5g45
            [50, 50, 25, 0, 25, 0, 25, 0], #5g45v2
            [50, 50, 30, 0, 30, 0, 30, 0], #5g45v3
            [130, 130, 20, 0,130, 0, 20, 0], #5g45v4
            [130, 130, 22, 0,130, 0, 22, 0], #5g45v4
            [130, 130, 25, 0,130, 0, 25, 0], #5g45v4
            [130, 130, 30, 0,130, 0, 30, 0], #5g45v4
            [116, 15, 18, 0, 18, 100, 3, 1], #start20
            [60, 10, 18, 25, 3, 25, 3, 1], #tapau20v1
            [40, 10, 18, 15, 3, 15, 3, 1], #tapau20v3
            [35, 5, 18, 15, 3, 15, 3, 1], #tapau20v4
            [56, 6, 18, 0, 18, 50, 3, 1], # BBNUv1
            [36, 6, 18, 0, 18, 30, 3, 1] # BBNUv2
    
           ]
df_new_plan = pd.DataFrame(new_plan, columns=core_variables)
df_possible_combinations = df_possible_combinations.append(df_new_plan, ignore_index=True)

In [80]:
# df_possible_combinations['zerorised_ind'] = [1 if fup == 0 or speed == 0 else 0 for fup, speed in zip(df_possible_combinations['main_bucket_fup'], df_possible_combinations['main_bucket_speed'])]
df_possible_combinations['very_low_fup_ind'] = [1 if fup <= 10 else 0 for fup in df_possible_combinations['total_fup']]
df_possible_combinations['low_fup_ind'] = [1 if fup <= 30 else 0 for fup in df_possible_combinations['total_fup']]
df_possible_combinations['high_fup_ind'] = [1 if fup >= 80 else 0 for fup in df_possible_combinations['total_fup']]
df_possible_combinations['separated_video_fup_ind'] = [1 if video_fup != 0 else 0 for video_fup in df_possible_combinations['video_fup']]
df_possible_combinations['separated_social_fup_ind'] = [1 if social_fup != 0 else 0 for social_fup in df_possible_combinations['social_fup']]
df_possible_combinations['slower_video_speed_ind'] = [1 if video_speed != 0 and video_speed < main_bucket_speed else 0 
                                    for video_speed, main_bucket_speed in zip(df_possible_combinations['video_speed'], df_possible_combinations['main_bucket_speed'])]

df_possible_combinations['const'] = 1.0
df_possible_combinations = df_possible_combinations[['const'] + variables]
df_possible_combinations['predicted_adpu'] = poisson_results.predict(df_possible_combinations)
df_possible_combinations

,const,main_bucket_fup,main_bucket_speed,video_fup,video_speed,social_fup,social_speed,unl_ind,very_low_fup_ind,low_fup_ind,high_fup_ind,separated_video_fup_ind,separated_social_fup_ind,slower_video_speed_ind,predicted_adpu
0,1.00,10,1,0.00,1,0.00,1,1.00,1,1,0,0,0,0,8.63
1,1.00,10,2,0.00,2,0.00,2,1.00,1,1,0,0,0,0,8.84
2,1.00,10,3,0.00,3,0.00,3,1.00,1,1,0,0,0,0,9.04
3,1.00,10,4,0.00,4,0.00,4,1.00,1,1,0,0,0,0,9.26
4,1.00,10,5,0.00,5,0.00,5,1.00,1,1,0,0,0,0,9.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,1.00,10,18,25.00,3,25.00,3,1.00,0,0,0,1,1,1,27.35
192,1.00,10,18,15.00,3,15.00,3,1.00,0,0,0,1,1,1,27.07
193,1.00,5,18,15.00,3,15.00,3,1.00,0,0,0,1,1,1,26.13
194,1.00,6,18,0.00,18,50.00,3,1.00,0,0,0,0,1,0,22.07


In [56]:
df_possible_combinations.to_csv(r'C:\Users\kianhao.chew\Desktop\repos\adpu-dash\predicted_adpu_v12.csv')